In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,65364.02,-0.89,-0.18,-0.08,0.00,-0.01,0.00,-0.00
3,-0.89,8393.76,0.06,-0.03,0.01,-0.00,-0.00,0.00
4,-0.18,0.06,2991.36,-0.01,0.02,0.01,-0.00,-0.00
5,-0.08,-0.03,-0.01,659.13,-0.00,-0.00,0.00,-0.00
6,0.00,0.01,0.02,-0.00,121.11,0.00,0.00,0.00
7,-0.01,-0.00,0.01,-0.00,0.00,20.53,-0.00,-0.00
8,0.00,-0.00,-0.00,0.00,0.00,-0.00,2.99,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.27


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00004,-0.00001,-0.00001,0.00000,-0.00001,0.00001,-0.00001
3,-0.00004,1.00000,0.00001,-0.00001,0.00001,-0.00001,-0.00001,0.00000
4,-0.00001,0.00001,1.00000,-0.00001,0.00004,0.00003,-0.00000,-0.00002
5,-0.00001,-0.00001,-0.00001,1.00000,-0.00001,-0.00001,0.00001,-0.00000
6,0.00000,0.00001,0.00004,-0.00001,1.00000,0.00001,0.00000,0.00003
7,-0.00001,-0.00001,0.00003,-0.00001,0.00001,1.00000,-0.00001,-0.00001
8,0.00001,-0.00001,-0.00000,0.00001,0.00000,-0.00001,1.00000,0.00002
9,-0.00001,0.00000,-0.00002,-0.00000,0.00003,-0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.298940736393716

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.34266514e+08 8.66046789e+06 1.06326293e+06 4.96512481e+04
 1.57325617e+03 4.02281369e+01 6.41162090e-01 1.70349787e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999804,-0.018651,-0.006535,-0.001437,-0.000264,-0.000045,-0.000006,-6.034567e-07
1,0.018147,0.997461,-0.067621,-0.012799,-0.002334,-0.000396,-0.000058,-5.127003e-06
2,0.007690,0.066797,0.996587,-0.047174,-0.008168,-0.001381,-0.000202,-1.817599e-05
3,0.002014,0.015751,0.045750,0.997649,-0.047853,-0.007779,-0.001132,-1.010812e-04
4,0.000460,0.003584,0.010064,0.046844,0.997107,-0.058320,-0.008182,-7.278500e-04
5,0.000104,0.000808,0.002265,0.010300,0.057101,0.995511,-0.074456,-6.432651e-03
6,0.000023,0.000177,0.000495,0.002248,0.012242,0.072979,0.993226,-8.956421e-02
7,0.000004,0.000032,0.000089,0.000403,0.002193,0.012949,0.088831,9.959600e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00019637736587230226,
 0.002538893399256281,
 0.0034133258206116013,
 0.00235122000235255,
 0.002892903589456597,
 0.004488825903942173,
 0.006774420272210979,
 0.004039993981585277]